In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [2]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 25.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 12.5 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.0 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039286 sha256=8c8e80e3213c96fc16954fc832809f442adf21d9092a153033a308daa8a51472
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
  Created wheel for umap-

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)

    filtered_text = [word for word in word_tokens if word.lower() not in stop_words]

    return ' '.join(filtered_text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import os

base_path = '/content/drive/MyDrive/bbc/'
# topics = ['tech','politics','sport','business','entertainment']
topics = ['politics']
contents = {}
for topic in topics:
  contents[topic] = []
  folder_path = base_path + topic
  # Get a list of all files in the folder
  all_files = os.listdir(folder_path)
  # Filter out only the files (excluding subdirectories)
  files_only = [file for file in all_files if os.path.isfile(os.path.join(folder_path, file))]
  for file in files_only:
      file_path = os.path.join(folder_path, file)
      with open(file_path, 'r', encoding='utf-8') as txt_file:
            file_content = txt_file.read()
            contents[topic].append(remove_stopwords(file_content))


In [5]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

top_topics = {}
for topic in topics:
  # Create BERTopic model
  model = BERTopic()
  topics, probabilities = model.fit_transform(contents[topic])
  # Get the top topics
  top_topic = model.get_topic_freq().head(5)  # Get the top 5 topics
  top_topics[topic] = {"topics":top_topic, "model":model}

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
politics = top_topics['politics']
# Print the top topics
print(politics["topics"])
politics["model"].get_topic_info()
# Visualize topics
# politics["model"].visualize_topics()

   Topic  Count
1      0    246
2      1     45
0     -1     43
3      2     16
7      3     15


,Topic,Count,Name,Representation,Representative_Docs
0,-1,43,-1_said_mr_blunkett_cards,"[said, mr, blunkett, cards, id, people, would,...",[Howard denies split ID cards Michael Howard d...
1,0,246,0_mr_said_labour_would,"[mr, said, labour, would, election, blair, bro...",[Brown appeal Labour unity Gordon Brown made a...
2,1,45,1_said_police_law_would,"[said, police, law, would, rights, home, human...",[Lords wrong detainees - Straw Jack Straw atta...
3,2,16,2_asylum_immigration_uk_said,"[asylum, immigration, uk, said, system, howard...",[Howard attacks cost asylum Michael Howard lau...
4,3,15,3_eu_straw_constitution_china,"[eu, straw, constitution, china, referendum, s...",[Straw attacked China arms Moves lift European...
5,4,14,4_hunting_ban_hunt_law,"[hunting, ban, hunt, law, said, hunts, dogs, p...",[Ban hunting comes force Fox hunting dogs ille...
6,5,13,5_lord_lords_speaker_house,"[lord, lords, speaker, house, said, mps, kinno...",['Best person ' top legal job `` best person j...
7,6,13,6_ukip_kilroysilk_party_mr,"[ukip, kilroysilk, party, mr, veritas, robert,...",[Kilroy launches 'Veritas ' party Ex-BBC chat ...
8,7,12,7_workers_union_unions_pension,"[workers, union, unions, pension, public, stri...",[Stalemate pension strike talks Talks aimed av...


In [7]:
# Get topic names
label_names = politics["model"].generate_topic_labels(nr_words=3, topic_prefix=True, word_length=7, separator='_')
# politics["model"].set_topic_labels(labels)
politics["model"].custom_labels_ = label_names
politics["model"].visualize_barchart(custom_labels=True)

In [9]:
df = politics["model"].get_document_info(contents['politics'])
df.head()

,Document,Topic,Name,CustomName,Representation,Representative_Docs,Top_n_words,Probability,Representative_document
0,Election 'could terror target' Terrorists migh...,-1,-1_said_mr_blunkett_cards,-1_said_mr_blunket,"[said, mr, blunkett, cards, id, people, would,...",[Howard denies split ID cards Michael Howard d...,said - mr - blunkett - cards - id - people - w...,0.000000,False
1,'Debate needed ' donations cap cap donations p...,0,0_mr_said_labour_would,0_mr_said_labour,"[mr, said, labour, would, election, blair, bro...",[Brown appeal Labour unity Gordon Brown made a...,mr - said - labour - would - election - blair ...,1.000000,False
2,Kennedy begins pre-election tour Liberal Democ...,0,0_mr_said_labour_would,0_mr_said_labour,"[mr, said, labour, would, election, blair, bro...",[Brown appeal Labour unity Gordon Brown made a...,mr - said - labour - would - election - blair ...,1.000000,False
3,'Errors ' doomed first Dome sale initial attem...,0,0_mr_said_labour_would,0_mr_said_labour,"[mr, said, labour, would, election, blair, bro...",[Brown appeal Labour unity Gordon Brown made a...,mr - said - labour - would - election - blair ...,0.911593,False
4,Correction agency plans dropped Plans create s...,1,1_said_police_law_would,1_said_police_law,"[said, police, law, would, rights, home, human...",[Lords wrong detainees - Straw Jack Straw atta...,said - police - law - would - rights - home - ...,0.659644,False


In [11]:
# import the OpenAI Python library for calling the OpenAI API
from openai import OpenAI
import os

client = OpenAI(api_key="")


In [20]:
politics_df = politics["model"].get_topic_info()

In [22]:
politics_df['label'] = ['' for x in range(politics_df.shape[0])]
politics_df.head()

,Topic,Count,Name,CustomName,Representation,Representative_Docs,label
0,-1,43,-1_said_mr_blunkett_cards,-1_said_mr_blunket,"[said, mr, blunkett, cards, id, people, would,...",[Howard denies split ID cards Michael Howard d...,
1,0,246,0_mr_said_labour_would,0_mr_said_labour,"[mr, said, labour, would, election, blair, bro...",[Brown appeal Labour unity Gordon Brown made a...,
2,1,45,1_said_police_law_would,1_said_police_law,"[said, police, law, would, rights, home, human...",[Lords wrong detainees - Straw Jack Straw atta...,
3,2,16,2_asylum_immigration_uk_said,2_asylum_immigra_uk,"[asylum, immigration, uk, said, system, howard...",[Howard attacks cost asylum Michael Howard lau...,
4,3,15,3_eu_straw_constitution_china,3_eu_straw_constit,"[eu, straw, constitution, china, referendum, s...",[Straw attacked China arms Moves lift European...,


In [24]:
politics_label = {}
for i, r in politics_df.iterrows():
    docs = r["Representative_Docs"]
    words = r["Representation"]

    txt = "Document: ".join(docs)
    words = ", ".join(words)

    prompt = """I have topic that contains the following documents: ----- \n """+txt+"""
    ------------
    The topic is described by the following keywords: """+words+"""

    Based on the above information, can you give a short label of the topic?
    Instructions: Keep the response crisp and brief, the label should contain less than or equal to 6 words in it.
    """
    MODEL = "gpt-3.5-turbo"
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=0,
    )
    politics_label[r['Topic']] = response.choices[0].message.content


In [25]:
politics_df['label'] = politics_df['Topic'].apply(lambda x:politics_label[x])

In [26]:
politics_df

,Topic,Count,Name,CustomName,Representation,Representative_Docs,label
0,-1,43,-1_said_mr_blunkett_cards,-1_said_mr_blunket,"[said, mr, blunkett, cards, id, people, would,...",[Howard denies split ID cards Michael Howard d...,Government debate on ID cards
1,0,246,0_mr_said_labour_would,0_mr_said_labour,"[mr, said, labour, would, election, blair, bro...",[Brown appeal Labour unity Gordon Brown made a...,Labour Party rift between Blair and Brown
2,1,45,1_said_police_law_would,1_said_police_law,"[said, police, law, would, rights, home, human...",[Lords wrong detainees - Straw Jack Straw atta...,Foreign terror suspects detention controversy.
3,2,16,2_asylum_immigration_uk_said,2_asylum_immigra_uk,"[asylum, immigration, uk, said, system, howard...",[Howard attacks cost asylum Michael Howard lau...,UK asylum and immigration reforms.
4,3,15,3_eu_straw_constitution_china,3_eu_straw_constit,"[eu, straw, constitution, china, referendum, s...",[Straw attacked China arms Moves lift European...,EU arms embargo on China debate
5,4,14,4_hunting_ban_hunt_law,4_hunting_ban_hunt,"[hunting, ban, hunt, law, said, hunts, dogs, p...",[Ban hunting comes force Fox hunting dogs ille...,Illegal hunting ban enforcement challenges
6,5,13,5_lord_lords_speaker_house,5_lord_lords_speaker,"[lord, lords, speaker, house, said, mps, kinno...",['Best person ' top legal job `` best person j...,House of Lords Speaker Reform
7,6,13,6_ukip_kilroysilk_party_mr,6_ukip_kilroys_party,"[ukip, kilroysilk, party, mr, veritas, robert,...",[Kilroy launches 'Veritas ' party Ex-BBC chat ...,"""UKIP defection to Kilroy's Veritas"""
8,7,12,7_workers_union_unions_pension,7_workers_union_unions,"[workers, union, unions, pension, public, stri...",[Stalemate pension strike talks Talks aimed av...,Pension strike talks in public sector


In [32]:
# Get topic names
label_names = list(politics_df['label'])
# politics["model"].set_topic_labels(labels)
politics["model"].custom_labels_ = label_names
politics["model"].visualize_barchart(custom_labels=True,width=370)

In [33]:
df['label'] = df['Topic'].apply(lambda x:politics_label[x])
df.head()

,Document,Topic,Name,CustomName,Representation,Representative_Docs,Top_n_words,Probability,Representative_document,label
0,Election 'could terror target' Terrorists migh...,-1,-1_said_mr_blunkett_cards,-1_said_mr_blunket,"[said, mr, blunkett, cards, id, people, would,...",[Howard denies split ID cards Michael Howard d...,said - mr - blunkett - cards - id - people - w...,0.000000,False,Government debate on ID cards
1,'Debate needed ' donations cap cap donations p...,0,0_mr_said_labour_would,0_mr_said_labour,"[mr, said, labour, would, election, blair, bro...",[Brown appeal Labour unity Gordon Brown made a...,mr - said - labour - would - election - blair ...,1.000000,False,Labour Party rift between Blair and Brown
2,Kennedy begins pre-election tour Liberal Democ...,0,0_mr_said_labour_would,0_mr_said_labour,"[mr, said, labour, would, election, blair, bro...",[Brown appeal Labour unity Gordon Brown made a...,mr - said - labour - would - election - blair ...,1.000000,False,Labour Party rift between Blair and Brown
3,'Errors ' doomed first Dome sale initial attem...,0,0_mr_said_labour_would,0_mr_said_labour,"[mr, said, labour, would, election, blair, bro...",[Brown appeal Labour unity Gordon Brown made a...,mr - said - labour - would - election - blair ...,0.911593,False,Labour Party rift between Blair and Brown
4,Correction agency plans dropped Plans create s...,1,1_said_police_law_would,1_said_police_law,"[said, police, law, would, rights, home, human...",[Lords wrong detainees - Straw Jack Straw atta...,said - police - law - would - rights - home - ...,0.659644,False,Foreign terror suspects detention controversy.


In [ ]:
df.to_csv("politics_topics.csv", index=False)